In [1]:
import os
import difflib
import zipfile
import lxml.etree as etree
import random
import re
import sys
from datetime import datetime
import pandas as pd

import django
import platform

sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import cities.models as cmodels
from django.contrib.auth.models import User
import tmv_app.models as tm

#from parsing_utils import POI, dehyphenate_with_space, clean_text, find_person_in_db_aph
#from aph_model_builder import *

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [2]:
data_dir = '/home/leey/Documents/Data/AustralianPoliticians/data-raw/'

In [3]:
df_all = pd.read_csv(data_dir+'all.csv')

In [58]:
df_all.head()

,uniqueID,surname,allOtherNames,firstName,commonName,displayName,earlierOrLaterNames,title,gender,birthDate,birthYear,deathDate,member,senator,wikidataID,wikipedia,adb,comments
0,Abbott1859,Abbott,Richard Hartley Smith,Richard,NaN,"Abbott, Richard",NaN,NaN,male,NaN,1859.0,1940-02-28,0,1,Q7323613,https://en.wikipedia.org/wiki/Richard_Abbott_(...,NaN,NaN
1,Abbott1869,Abbott,Percy Phipps,Percy,NaN,"Abbott, Percy",NaN,NaN,male,1869-05-14,NaN,1940-09-09,1,1,Q7167212,https://en.wikipedia.org/wiki/Percy_Abbott_(Au...,NaN,NaN
2,Abbott1877,Abbott,Macartney,Macartney,Mac,"Abbott, Mac",NaN,NaN,male,1877-07-03,NaN,1960-12-30,0,1,Q6722175,https://en.wikipedia.org/wiki/Mac_Abbott,NaN,NaN
3,Abbott1886,Abbott,Charles Lydiard Aubrey,Charles,Aubrey,"Abbott, Aubrey",NaN,NaN,male,1886-01-04,NaN,1975-04-30,1,0,Q4819077,https://en.wikipedia.org/wiki/Aubrey_Abbott,NaN,NaN
4,Abbott1891,Abbott,Joseph Palmer,Joseph,NaN,"Abbott, Joseph",NaN,NaN,male,1891-10-18,NaN,1965-05-07,1,0,Q6286049,https://en.wikipedia.org/wiki/Joseph_Palmer_Ab...,NaN,NaN


In [4]:
df_all['deathDate'] = pd.to_datetime(df_all['deathDate'],infer_datetime_format=True)

In [5]:
df_all['birthDate'] = pd.to_datetime(df_all['birthDate'],infer_datetime_format=True)

In [6]:
df_all.dtypes

uniqueID                       object
surname                        object
allOtherNames                  object
firstName                      object
commonName                     object
displayName                    object
earlierOrLaterNames            object
title                          object
gender                         object
birthDate              datetime64[ns]
birthYear                     float64
deathDate              datetime64[ns]
member                          int64
senator                         int64
wikidataID                     object
wikipedia                      object
adb                            object
comments                       object
dtype: object

In [68]:
df_all['birthDate'][0] == None

False

In [15]:
for i in df_all.index:
    pol = df_all.loc[i]
    break
title = str(pol['title'])
type(title)

str

In [30]:
import numpy as np

In [29]:
for i in df_all.index:
    pol = df_all.loc[i]
    break
title = pol['birthDate']
type(title)

float

In [31]:
np.isnan(title)

True

In [5]:
df_all.columns

Index(['uniqueID', 'surname', 'allOtherNames', 'firstName', 'commonName',
       'displayName', 'earlierOrLaterNames', 'title', 'gender', 'birthDate',
       'birthYear', 'deathDate', 'member', 'senator', 'wikidataID',
       'wikipedia', 'adb', 'comments'],
      dtype='object')

In [6]:
df_party = pd.read_csv(data_dir+'by_party.csv')

In [7]:
df_party.head()

,uniqueID,partyAbbrev,partyName,partyFrom,partyTo,partyChangedName,partySimplifiedName,partySpecificDateInputted,partyComments
0,Abbott1859,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
1,Abbott1869,CP,Australian Country Party,1919-11-03,NaN,NaN,Nationals,NaN,NaN
2,Abbott1869,LIB,Liberal Party of Australia,NaN,1917-02-17,NaN,Liberals,NaN,NaN
3,Abbott1869,NAT,Nationalist Party,1917-02-17,1919-11-03,NaN,Nationalist Party,NaN,NaN
4,Abbott1877,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN


In [12]:
for i in df_all.index:
    pol = df_all.loc[i]
    df_party.where(df_party['uniqueID']==pol['uniqueID']).notna()

### building a parser

In [3]:
POL_FNAME = data_dir + 'all.csv'

In [32]:
def parse_aph_data(verbosity=0):

    warn = 0

    df_pol = pd.read_csv(POL_FNAME)
    print("read data from {}".format(POL_FNAME))

    # going through entries for mdbs
    for i in df_pol.index:
        pol = df_pol.loc[i]

        person = pm.Person(
            surname=str(pol['surname']),
            first_name=str(pol['firstName']),
            unique_id=str(pol['uniqueID'])
            )

        # profile information
        title = str(pol['title'])
        if title is not None:
            person.title = title

        dob = pol['birthDate']
        dod = pol['deathDate']
        yob = int(pol['birthYear'])
        
        if np.isnan(dob) == False:
            person.date_of_birth = datetime.strptime(str(dob), '%Y-%m-%d') 
        else:
            person.year_of_birth = yob
        
        if np.isnan(dod) == False:
            person.date_of_death = datetime.strptime(str(dod), '%Y-%m-%d')

        if pol['gender'] == "male":
            person.gender = pm.Person.MALE
        elif pol['gender'] == "female":
            person.gender = pm.Person.FEMALE

        # additional names
        # surnames_list = []
        firstnames_list = []

        firstname = pol['allOtherNames']

        if isinstance(firstname, str):
            firstnames_list.append(firstname)

            if len(firstname.split(' ')) > 1:
                for fname in firstname.split(" "):
                    if len(fname) > 2:
                        firstnames_list.append(fname)


        commonname = pol['commonName']

        if isinstance(commonname, str):
            firstnames_list.append(commonname)

            if len(commonname.split(' ')) > 1:
                for cname in commonname.split(" "):
                    if len(cname) > 2:
                        firstnames_list.append(cname)

        person.alt_first_names = list(set(firstnames_list))

        if verbosity > 0:
            # print name
            print("Person: {}".format(person))

        # political party
        # match politician ID in df_pol to df_party to find party
        #try:
        #    person.party = Party.objects.get(alt_names__contains=)
        #except Party.DoesNotExist:
        #    if biodata.find('PARTEI_KURZ').text == 'Plos':
        #        person.party = Party.objects.get(name="parteilos")
        #        pass
        #    else:
        #        print("Warning: Party not found: {}".format(biodata.find("PARTEI_KURZ").text))
        #        warn += 1

        # name id


        person.active_country = cmodels.Country.objects.get(name='Australia')
        # adding positions
        if pol['member'] == 1:
            person.positions = ['Member']
        if pol['senator'] == 1:
            person.positions = ['Senator']

        person.information_source = "AustralianPoliticians"
        person.save()

        person.in_parlperiod = wp_list
        person.save()

    print("Done. {} warnings.".format(warn))

In [33]:
if __name__ == '__main__':
    delete_all = False
    delete_persons = False

    if delete_all:
        # to delete all existing entries for constituencies, partylists, seats and persons
        pm.Constituency.objects.all().delete()
        pm.PartyList.objects.all().delete()
        pm.Party.objects.all().delete()
        pm.Seat.objects.all().delete()
        pm.Person.objects.all().delete()

    elif delete_persons:
        pm.Person.objects.filter(information_source="AustralianPoliticians").delete()

    # updating the parties
    # update_parties()
    # parsing the data
    parse_aph_data(verbosity=0)

read data from /home/leey/Documents/Data/AustralianPoliticians/data-raw/all.csv


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

### just testing

In [8]:
warn = 0

#df_pol = pd.read_csv(POL_FNAME)
#print("read data from {}".format(POL_FNAME))

df_pol = df_all

# going through entries for mdbs
for i in df_pol.index:
    pol = df_pol.loc[i]

    person = pm.Person(
        surname=str(pol['surname']),
        first_name=str(pol['firstName']),
        unique_id=str(pol['uniqueID'])
        )

    # profile information
    title = str(pol['title'])
    if title is not None:
        person.title = title

    dob = pol['birthDate']
    dod = pol['deathDate']
    yob = int(pol['birthYear'])

    if dob == False:
        person.date_of_birth = datetime.strptime(str(dob), '%Y-%m-%d') 
    else:
        person.year_of_birth = yob

    if dod == False:
        person.date_of_death = datetime.strptime(str(dod), '%Y-%m-%d')

    if pol['gender'] == "male":
        person.gender = pm.Person.MALE
    elif pol['gender'] == "female":
        person.gender = pm.Person.FEMALE

    # additional names
    # surnames_list = []
    firstnames_list = []

    firstname = pol['allOtherNames']

    if isinstance(firstname, str):
        firstnames_list.append(firstname)

        if len(firstname.split(' ')) > 1:
            for fname in firstname.split(" "):
                if len(fname) > 2:
                    firstnames_list.append(fname)


    commonname = pol['commonName']

    if isinstance(commonname, str):
        firstnames_list.append(commonname)

        if len(commonname.split(' ')) > 1:
            for cname in commonname.split(" "):
                if len(cname) > 2:
                    firstnames_list.append(cname)

    person.alt_first_names = list(set(firstnames_list))

    #if verbosity > 0:
        # print name
    #    print("Person: {}".format(person))

    person.active_country = cmodels.Country.objects.get(name='Australia')
    # adding positions
    if pol['member'] == 1:
        person.positions = ['Member']
    if pol['senator'] == 1:
        person.positions = ['Senator']

    person.information_source = "AustralianPoliticians"
    person.save()

    #person.in_parlperiod = wp_list
    person.save()

    break

print("Done. {} warnings.".format(warn))

Done. 0 warnings.
